# Overview

This tutorial focuses on the modelling of polymers with ellipsoids. The concept of ellipsoid modelling is very similar to the spheroid coarse graining in [tutorial 3](./3-flowerMD-coarse-graining.ipynb), but using ellipsoids instead of spheres to model monomers.

The utility of ellipsoids over standard coarse graining is to better model shapes that are not spherical. For example, PPS is a very flat molecule, and would be better modelled by an oblate ellipsoid than a sphere.

Readers of this tutorial should learn how to:
- Initialize a system of ellipsoid polymers
- Create a rigid body constraint for an ellipsoid chain
- Visualize a system of polymers

In [1]:
import warnings

warnings.filterwarnings("ignore")
import hoomd
import unyt as u
from flowermd.base import Pack, Simulation
from flowermd.library import EllipsoidChain
from flowermd.library import EllipsoidForcefield
from flowermd.utils.constraints import create_rigid_ellipsoid_chain
from flowermd.utils import get_target_box_number_density
from cmeutils.gsd_utils import ellipsoid_gsd

## System Initialization

Ellipsoids are represented by the `EllipsoidChain` class. It can be treated like a standard `Molecule`.

The ellipsoidal geometry is defined by two parameters: `lpar` and `lperp`. These represent the major and minor axis lengths, respectively. `lpar` > `lperp` will create a prolate ellipsoid, and `lpar` < `lperp` will create an oblate ellipsoid. `lpar` = `lperp` will create a sphere.

`bead_mass` defines the mass of each monomer in the polymer.

In [2]:
LPAR = 1.0
LPERP = 0.5
BOND_L = 0.05

GSD_PATH = "traj.gsd"

ellipsoid_chain = EllipsoidChain(
    lengths=5, num_mols=5, lpar=LPAR, bead_mass=1.0
)
system = Pack(
    molecules=ellipsoid_chain,
    density=0.01 * u.Unit("nm**-3"),
    packing_expand_factor=6,
    edge=2,
    overlap=2,
    fix_orientation=True,
)

## Forcefield definition

The `EllipsoidForcefield` class is used to represent the forces between each ellipsoid, and uses the Gay-Berne forcefield, which is similar to a Lennard-Jones forcefield, but takes the orientation and shape of an ellipsoid into account.

Epsilon defines how deep the energy well for the Gay-Berne potential graph should be.

In [3]:
ff = EllipsoidForcefield(
    epsilon=1.0,
    lpar=LPAR,
    lperp=LPERP,
    r_cut=2.0,
    bond_k=100,
    bond_r0=0,
    angle_k=30,
    angle_theta0=1.9,
)

## Rigid Body Constraint

The construction of each ellipsoidal monomer consists of a center ('X' particle), bond ('A' particle), head ('T' particle), and tail (also 'T' particle) all assembled into one compound. The `create_rigid_ellipsoid_chain` function defines a frame and a constraint to be supplied to the simulation that will ensure the body does not change shape. The `lperp` parameter in this function is necessary to calculate the moment of inertia for the particles in the supplied frame.

The `rigid_frame` variable is identical to `system.hoomd_snapshot`, but with each monomer replaced with a rigid body ellipsoid.

The `rigid_constraint` variable is a constraint that is to be passed into the simulation to ensure that each body remains rigid throughout the simulation.

In [4]:
rigid_frame, rigid_constraint = create_rigid_ellipsoid_chain(
    system.hoomd_snapshot, LPERP
)

TypeError: create_rigid_ellipsoid_chain() takes 1 positional argument but 2 were given

## Simulation

The simulation of ellipsoids behaves largely identically to other simulations that have been introduced thus far.

In [ ]:
ellipsoid_sim = Simulation(
    initial_state=rigid_frame,
    forcefield=ff.hoomd_forces,
    constraint=rigid_constraint,
    dt=0.001,
    gsd_write_freq=int(1e4),
    gsd_file_name=GSD_PATH,
    log_write_freq=int(1e4),
    log_file_name="ellipsoid.txt",
)

target_box = get_target_box_number_density(
    density=0.03 * u.Unit("nm**-3"), n_beads=25
)
ellipsoid_sim.run_update_volume(
    final_box_lengths=target_box,
    kT=1.0,
    n_steps=int(5e4),
    tau_kt=10 * ellipsoid_sim.dt,
    period=10,
    thermalize_particles=True,
)
print("shrink finished")
ellipsoid_sim.run_NVT(n_steps=1e4, kT=1.0, tau_kt=10 * ellipsoid_sim.dt)
ellipsoid_sim.save_restart_gsd("restart.gsd")
ellipsoid_sim.flush_writers()
ellipsoid_sim.save_simulation("sim.pickle")

## Restarting Simulation with Rigid Constraint

When restarting a simulation, one must still apply the rigid constraint. However, the rigid constraint used previously cannot be reused. Thus, this function can be used to create a new rigid constraint

In [ ]:
def restart_rigid_ellipsoid() -> hoomd.md.constrain.Rigid:
    local_coords = [
        (0.0, 0.0, 0.0),
        (LPAR + BOND_L, 0.0, 0.0),
        (LPAR, 0.0, 0.0),
        (-LPAR, 0.0, 0.0),
    ]
    rigid_constrain = hoomd.md.constrain.Rigid()
    rigid_constrain.body["R"] = {
        "constituent_types": ["X", "A", "T", "T"],
        "positions": local_coords,
        "orientations": [[1, 0, 0, 0]] * len(local_coords),
    }
    return rigid_constrain

In [ ]:
ff = EllipsoidForcefield(
    epsilon=1.0,
    lpar=LPAR,
    lperp=LPERP,
    r_cut=2.0,
    bond_k=100,
    bond_r0=0,
    angle_k=30,
    angle_theta0=1.9,
)

rigid = restart_rigid_ellipsoid()
gsd_path = "sim_restart.gsd"

sim = Simulation(
    initial_state="restart.gsd",
    forcefield=ff.hoomd_forces,
    constraint=rigid,
    gsd_write_freq=int(1e3),
    gsd_file_name=gsd_path,
    log_write_freq=int(1e4),
    log_file_name="log2.txt",
)
sim.run_NVT(n_steps=1e5, kT=1.0, tau_kt=10 * sim.dt)
sim.flush_writers()

## Adding ellipsoid information to GSD

By default, the GSD file will not include any information about the ellipsoids themselves, and when you attempt to visualize the ellipsoid, each monomer will simply appear as 4 spheres due to how the body was constructed (see [Rigid Body Constraint](#Rigid-Body-Constraint)). In order to embed the geometrical information about the ellipsoid into a new GSD file, one can use the `ellipsoid_gsd` function from `cmeutils.gsd_utils`.

The new file that has all of the data one needs to visualize is supplied in the `new_file` argument. In this case, 'visual-ellipsoid.gsd' will contain the correct data.

The `ellipsoid_types` argument tells `ellipsoid_gsd` which particles to embed the ellipsoid information for. For ellipsoids, we use 'R', which represents the rigid body for every monomer.

One can visualize ellipsoids by opening "visual-ellipsoid.gsd" inside of a visualization tool such as [OVITO](https://www.ovito.org/)

In [ ]:
ellipsoid_gsd(
    gsd_file=GSD_PATH,
    new_file="visual-ellipsoid.gsd",
    ellipsoid_types="R",
    lpar=LPAR,
    lperp=LPERP,
)